In [84]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdFMCS
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import SimilarityMaps
from rdkit.Chem import Lipinski
from rdkit.Chem import Descriptors
import os
import re

In [34]:
#Wczytaj testowy plik
zinc = Chem.SmilesMolSupplier('zinc100.smi', titleLine=False)

#Calosc polaczona w petla, ktora pozwoli
for index in range(len(zinc)):
    if Chem.Lipinski.NumHAcceptors(zinc[index]) >= 3 \
    and Chem.Lipinski.NumHDonors(zinc[index]) >= 3 \
    and Chem.Lipinski.NumRotatableBonds(zinc[index]) >= 3 \
    and Chem.Descriptors.MolWt (zinc[index]) < 300 \
    and Chem.Descriptors.MolLogP (zinc[index]) < 3:
        zincID = zinc[index].GetProp('_Name')
        out = Chem.MolToSmiles(zinc[index])
        print(out,zincID, file=open('ro3_pass.smi', 'a+'))

In [35]:
from rdkit import DataStructs 
from rdkit.Chem.Fingerprints import FingerprintMols

#ze smilesow tworze mole
smiles_amc = Chem.SmilesMolSupplier('AMCase_smiles.smi')
smiles_zinc = Chem.SmilesMolSupplier('pass.smi')

In [36]:
# Stworzenie dla obu zbiorow fp Morgana

ecfp_amcase = [AllChem.GetMorganFingerprint(x , 2) for x in smiles_amc]
ecfp_zinc = [AllChem.GetMorganFingerprint(x , 2) for x in smiles_zinc]

In [62]:
#Wybierz z kazdego przeszukania 4 najbardziej podobne zwiazki
sim_dict = {} 
for idx in range(len(ecfp_amcase)):
    inner_dict = {}
    for idx_zinc in range(len(ecfp_zinc)):
        sim_morg = DataStructs.DiceSimilarity(ecfp_amcase[idx],ecfp_zinc[idx_zinc])
        # arbitrnalnie ustalam prob podobienstwa na 0.40
        if sim_morg >= 0.45: 
                inner_dict[idx_zinc] = sim_morg
    if inner_dict:
        sim_dict[idx] = sorted(inner_dict.items(), key = lambda x: x[1], reverse=True)
        sim_dict[idx] = sim_dict[idx][0:4]   

In [65]:
# robie zbior z identyfikatorow zwiazkow
idx_set = set()
for value in sim_dict.values():
    for idx, score in value:
        idx_set.add(idx)
print(len(idx_set))
        

53


In [69]:
# tworze plik smiles ze zwiazkow znalezionych wyzej
zinc_pass = Chem.SmilesMolSupplier('pass.smi')
for i in idx_set:
    zincID = zinc_pass[i].GetProp('_Name')
    out = Chem.MolToSmiles(zinc_pass[i])
    print(out,zincID, file=open('found_zinc.smi', 'a+')) 

In [149]:
# wyrzucam powtorzenia z utworzonego pliku smiles (identyfikatory lub struktury)
with open('found_zinc.smi', 'r') as f:
    structure_list = []
    name_list = []
    cleared_list = []
    doubling_list_structure = []
    doubling_list_name = []
    
    for row in f:
        structure_list.append(row.split()[0])
        name_list.append(row.split()[1])

    for idx in range(len(structure_list)):
        if structure_list[idx] not in doubling_list_structure \
        and structure_list[idx] not in structure_list[:idx] \
        and structure_list[idx] not in structure_list[idx+1:]\
        and name_list[idx] not in doubling_list_name \
        and name_list[idx] not in name_list[:idx] \ 
        and name_list[idx] not in name_list[idx+1:]:
            cleared_list.append([structure_list[idx], name_list[idx]])
            print(structure_list[idx], name_list[idx], file=open('cleared_zinc.smi', 'a+'))
        else: 
            doubling_list_structure.append(structure_list[idx])
            doubling_list_name.append(name_list[idx])


In [89]:
# Robie slownik najlepszych wynikow z dokowania

dir_path = os.getcwd() + '/wyniki_dokowania'
pdbqt_files = [file for file in os.listdir(dir_path) if '.pdbqt' in file]

vina_scores = {} 
for f in pdbqt_files:
    with open(f'{dir_path}/{f}', 'r') as f:
        f = f.read()
        score = float(re.findall('REMARK VINA RESULT: (.*)',f)[0].split()[0])
        zinc_id = re.findall('ZINC.*', f)[0]
        vina_scores[zinc_id] = score
vina_scores = dict(sorted(vina_scores.items(), key = lambda x: x[1]))
vina_scores

{'ZINC69869800': -9.5,
 'ZINC75567398': -9.1,
 'ZINC70451801': -9.1,
 'ZINC82766137': -8.8,
 'ZINC38032566': -8.6,
 'ZINC35476207': -8.5,
 'ZINC00184014': -8.4,
 'ZINC71493215': -8.1,
 'ZINC49845659': -8.1,
 'ZINC58393727': -8.0,
 'ZINC54257519': -8.0,
 'ZINC82752718': -7.9,
 'ZINC82766023': -7.9,
 'ZINC82752840': -7.9,
 'ZINC71630114': -7.8,
 'ZINC71588959': -7.7,
 'ZINC75973400': -7.5,
 'ZINC15636911': -7.4,
 'ZINC82807662': -7.2,
 'ZINC20266675': -6.8,
 'ZINC95915750': -6.4,
 'ZINC23253598': -6.3,
 'ZINC71601416': -6.1,
 'ZINC21987165': -6.0}

In [152]:
# Robię tabele rescoringu z wartosciami z poszczegolnych funkcji podanymi w poleceniu (RF-Score v2, NN-Score
# oraz PLEC Linear). Nastepnie na podstawie tej tabeli stworze listy rankingowe

import pandas as pd

dir_path_sdf = os.getcwd() + '/wynik_oddt_cli_1'
file_list = [file for file in os.listdir(dir_path_sdf) if '.sdf' in file]
score_dict = {'rfscore_v2': [],'pleclinear': [], 'nnscore': [], 'id': []}

for f in file_list:
    with open(f'{dir_path_sdf}/{f}', 'r') as file:
        file = file.read()
        score_dict['rfscore_v2'].append(max([float(val) for val in re.findall('<rfscore_v2>\n(.*)', file)]))
        score_dict['pleclinear'].append(max([float(val) for val in re.findall('<PLEClinear_p5_l1_s65536>\n(.*)', file)]))
        score_dict['nnscore'].append(max([float(val) for val in re.findall('<nnscore>\n(.*)', file)]))
        score_dict['id'].append(re.findall('(ZINC.*)', file)[0])

result_df = pd.DataFrame(score_dict) 
result_df

,rfscore_v2,pleclinear,nnscore,id
0,5.783795,2.238644,6.473338,ZINC71601416
1,6.711988,3.001759,6.518526,ZINC00184014
2,6.850936,3.804733,6.182069,ZINC35476207
3,6.677358,2.099990,7.018414,ZINC82766137
4,6.745519,1.884628,7.294286,ZINC82766023
5,6.783958,2.752657,6.626318,ZINC15636911
6,6.164471,2.762075,6.657730,ZINC54257519
7,6.482729,1.874223,6.918990,ZINC71630114
8,7.030438,3.111804,6.790497,ZINC82752718
9,5.823402,2.695312,5.572108,ZINC23253598


In [141]:
#robie ranking dla rfscore_v2
result_df['rank_rfscore_v2'] = result_df['rfscore_v2'].rank()
result_df.sort_values('rfscore_v2', inplace=True) 
result_df

,rfscore_v2,pleclinear,nnscore,id,rank_rfscore_v2
20,5.685099,2.008218,5.417990,ZINC21987165,1.0
0,5.783795,2.238644,6.473338,ZINC71601416,2.0
9,5.823402,2.695312,5.572108,ZINC23253598,3.0
14,5.937498,2.783379,5.895942,ZINC95915750,4.0
6,6.164471,2.762075,6.657730,ZINC54257519,5.0
15,6.173980,3.060176,6.933846,ZINC82752840,6.0
17,6.356233,3.250369,7.014964,ZINC71588959,7.0
19,6.367590,3.062315,6.606727,ZINC38032566,8.0
12,6.397128,1.677371,6.740751,ZINC58393727,9.0
13,6.437621,3.193293,6.709529,ZINC20266675,10.0


In [142]:
#robie ranking dla nnscore
result_df['rank_nnscore'] = result_df['nnscore'].rank()
result_df.sort_values('nnscore', inplace=True)
result_df

,rfscore_v2,pleclinear,nnscore,id,rank_rfscore_v2,rank_nnscore
20,5.685099,2.008218,5.417990,ZINC21987165,1.0,1.0
9,5.823402,2.695312,5.572108,ZINC23253598,3.0,2.0
14,5.937498,2.783379,5.895942,ZINC95915750,4.0,3.0
22,6.535955,3.207885,6.090204,ZINC82807662,13.0,4.0
2,6.850936,3.804733,6.182069,ZINC35476207,22.0,5.0
16,6.545745,2.561652,6.451933,ZINC71493215,14.0,6.0
0,5.783795,2.238644,6.473338,ZINC71601416,2.0,7.0
1,6.711988,3.001759,6.518526,ZINC00184014,17.0,8.0
19,6.367590,3.062315,6.606727,ZINC38032566,8.0,9.0
5,6.783958,2.752657,6.626318,ZINC15636911,21.0,10.0


In [143]:
#robie ranking dla pleclinear
result_df['rank_pleclinear'] = result_df['pleclinear'].rank()
result_df.sort_values('pleclinear', inplace=True)
result_df

,rfscore_v2,pleclinear,nnscore,id,rank_rfscore_v2,rank_nnscore,rank_pleclinear
12,6.397128,1.677371,6.740751,ZINC58393727,9.0,15.0,1.0
7,6.482729,1.874223,6.918990,ZINC71630114,11.0,17.0,2.0
4,6.745519,1.884628,7.294286,ZINC82766023,19.0,23.0,3.0
23,6.728265,1.951167,7.172641,ZINC75567398,18.0,22.0,4.0
20,5.685099,2.008218,5.417990,ZINC21987165,1.0,1.0,5.0
3,6.677358,2.099990,7.018414,ZINC82766137,16.0,21.0,6.0
0,5.783795,2.238644,6.473338,ZINC71601416,2.0,7.0,7.0
10,6.877614,2.470034,6.920618,ZINC70451801,23.0,18.0,8.0
21,6.574770,2.557977,6.661116,ZINC49845659,15.0,13.0,9.0
16,6.545745,2.561652,6.451933,ZINC71493215,14.0,6.0,10.0


In [144]:
#Wybieram związki wysoko ocenione przez wszystkie funkcje
result_df = result_df.loc[result_df['rank_rfscore_v2'] > 10] 
result_df = result_df.loc[result_df['rank_nnscore'] > 10]
result_df = result_df.loc[result_df['rank_pleclinear'] > 10]
result_df

,rfscore_v2,pleclinear,nnscore,id,rank_rfscore_v2,rank_nnscore,rank_pleclinear
11,6.771326,2.724648,7.334290,ZINC69869800,20.0,24.0,12.0
18,6.513607,3.032775,6.629687,ZINC75973400,12.0,11.0,17.0
8,7.030438,3.111804,6.790497,ZINC82752718,24.0,16.0,20.0


**Które związki zostały ocenione wysoko przez wszystkie funkcję oceny?**


Wysoko przez wszysykie funkcje oceny zostały ocenione związki w tabeli powyżej, tzn:
1. ZINC69869800
2. ZINC75973400	
3. ZINC82752718

Zgodnie ze wskazówkami, dla każdej funkcji zbudowałem listę rankingową, od 1, gdzie 1 to najgorszy wyniki, do 24, gdzie 24 to związek, który dostał najwyższy wynik w danej funkcji. Postanowiłem, że "wysoko ocenione" to znaczy, że związek dostał w ramach każdej funkcji wynik rankingu powyżej 10/24.

**Które związki dały najlepsze wyniki dokowania molekularnego?**


Poniżej widać część słownika z kolejno identyfikatorami związków i ich wynikami dokowania molekularnego (im mniejsze tym lepsze):

{'ZINC69869800': -9.5,
 'ZINC75567398': -9.1,
 'ZINC70451801': -9.1,
 'ZINC82766137': -8.8,
 'ZINC38032566': -8.6,
 'ZINC35476207': -8.5,
 'ZINC00184014': -8.4,}
 
Jak widać ZINC69869800 - czyli związek z najwyższym wynikiem dokowania molekularnego, został również wysoko oceniony przez wszystkie funkcje scorujące. Pozostałe 2 cząsteczki wysoko ocenione przez funkcje scorujące w listach rankingowe nie znalazły się w czołówce jeżeli chodzi o wyniki dokowania molekularnego.